# Questions MongoDB

In [81]:
# Import des packages nécessaires à l'exécution du notebook

from pymongo import MongoClient
from dotenv import load_dotenv
import os
import pymongo

load_dotenv()
ATLAS_KEY = os.getenv("ATLAS_KEY")


In [84]:
# Connexion à la base de données et à la collection utilisée

client = MongoClient(ATLAS_KEY)
db = client.IMDBscrap
collection = db.films

## Quel est le film le plus long ?


In [85]:
film_le_plus_long = collection.find_one(sort=[("duree", -1)])
film_le_plus_long['titre']

'Gone with the Wind'

## Quels sont les 5 films les mieux notés ?

In [86]:
results = collection.find(sort=[("score",-1)], limit=5)
for result in results:
    print(result['titre'], result['score'])


The Shawshank Redemption 9.3
The Godfather 9.2
The Lord of the Rings: The Return of the King 9.0
The Godfather Part II 9.0
The Dark Knight 9.0


## Dans combien de films a joué Morgan Freeman ? Tom Cruise ?


In [87]:
films_morgan_freeman = collection.count_documents({"acteurs_principaux":"Morgan Freeman"})
films_tom_cruise = collection.count_documents({"acteurs_principaux":"Tom Cruise"})

print(f"Morgan Freeman a joué dans {films_morgan_freeman} films de la liste et Tom Cruise a joué dans {films_tom_cruise} film de la liste.")


Morgan Freeman a joué dans 4 films de la liste et Tom Cruise a joué dans 1 film de la liste.


## Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [88]:
titres_horreur = collection.find({"genre":"Horror"},sort=([("score", -1)]),limit=3)
for titre in titres_horreur:
    print(titre['titre'])

Psycho
Alien
The Shining


In [89]:
titres_dramatiques = collection.find({"genre":"Drama"},sort=([("score", -1)]),limit=3)
for titre in titres_dramatiques:
    print(titre['titre'])

The Shawshank Redemption
The Godfather
The Dark Knight


In [90]:
titres_comiques = collection.find({"genre":"Comedy"},sort=([("score", -1)]),limit=3)
for titre in titres_comiques:
    print(titre['titre'])

Life Is Beautiful
The Intouchables
Modern Times


## Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [91]:
# Extraire les 100 films les mieux notés
top_films = collection.find(sort=[("score", -1)], limit=100)

# Initilisés un compteur pour chaque pays
nb_films_americains = 0
nb_films_francais = 0

# Compter le nombre de films par pays 
for film in top_films:
    if film["pays"] == "United States":
        nb_films_americains += 1
    elif film["pays"] == "France":
        nb_films_francais += 1
        
# Calculer les pourcentages
pourcentage_films_americains = (nb_films_americains / 100) * 100
pourcentage_films_francais = (nb_films_francais / 100) * 100

print("Pourcentage de films américains: ", pourcentage_films_americains)
print("Pourcentage de films français: ", pourcentage_films_francais)



Pourcentage de films américains:  60.0
Pourcentage de films français:  4.0


## Quel est la durée moyenne d’un film en fonction du genre ?


In [92]:
pipeline = [
    {"$unwind": "$genre"}, # Sépare en plusieurs documents les films qui ont plusieurs genres
    {"$group": {"_id": "$genre", "avg_duration": {"$avg": "$duree"}}} # Groupe par genre et calcule la durée moyenne
]

results = collection.aggregate(pipeline)

for result in results:
    print("Le genre {} a une durée moyenne de {:.2f} minutes.".format(result["_id"], result["avg_duration"]))



Le genre Crime a une durée moyenne de 132.50 minutes.
Le genre Western a une durée moyenne de 153.86 minutes.
Le genre War a une durée moyenne de 140.05 minutes.
Le genre Animation a une durée moyenne de 100.96 minutes.
Le genre Adventure a une durée moyenne de 130.12 minutes.
Le genre Fantasy a une durée moyenne de 123.57 minutes.
Le genre Thriller a une durée moyenne de 123.88 minutes.
Le genre Musical a une durée moyenne de 103.00 minutes.
Le genre Mystery a une durée moyenne de 124.61 minutes.
Le genre Romance a une durée moyenne de 121.82 minutes.
Le genre Drama a une durée moyenne de 134.50 minutes.
Le genre History a une durée moyenne de 152.11 minutes.
Le genre Biography a une durée moyenne de 145.50 minutes.
Le genre Film-Noir a une durée moyenne de 110.00 minutes.
Le genre Comedy a une durée moyenne de 106.98 minutes.
Le genre Action a une durée moyenne de 136.30 minutes.
Le genre Horror a une durée moyenne de 120.60 minutes.
Le genre Sci-Fi a une durée moyenne de 131.70 minu

## En fonction du genre, afficher la liste des films les plus longs.


In [95]:
# Extraire la liste des genres 
genres = collection.distinct("genre")

for genre in genres:
    resultats = collection.find({"genre": genre},sort=[("duree",-1)]).limit(3)
    print('########')
    print(genre," : ")
    for resultat in resultats:
        print(resultat["titre"])

########
Action  : 
Seven Samurai
The Lord of the Rings: The Return of the King
Avengers: Endgame
########
Adventure  : 
Lawrence of Arabia
Ben-Hur
The Lord of the Rings: The Return of the King
########
Animation  : 
Princess Mononoke
Spirited Away
Howl's Moving Castle
########
Biography  : 
Lawrence of Arabia
Schindler's List
The Wolf of Wall Street
########
Comedy  : 
The Wolf of Wall Street
3 Idiots
Green Book
########
Crime  : 
Once Upon a Time in America
The Godfather Part II
The Green Mile
########
Drama  : 
Gone with the Wind
Once Upon a Time in America
Lawrence of Arabia
########
Family  : 
The Sound of Music
Like Stars on Earth
Harry Potter and the Deathly Hallows: Part 2
########
Fantasy  : 
The Green Mile
Spider-Man: No Way Home
Pirates of the Caribbean: The Curse of the Black Pearl
########
Film-Noir  : 
Rebecca
Sunset Blvd.
Double Indemnity
########
History  : 
Schindler's List
Braveheart
The Great Escape
########
Horror  : 
The Shining
The Exorcist
Alien
########
Music  :

## En fonction du genre, quel est le coût de tournage d’une minute de film ?

In [97]:
# Récupération des genres présents dans la collection MongoDB
genres = collection.distinct("genre")

# Boucle sur tous les genres pour calculer le coût de tournage d'une minute de film
for genre in genres:
    # Pipeline de requête MongoDB pour calculer le coût de tournage d'une minute de film pour le genre en cours de traitement
    pipeline = [
    {"$unwind": "$genre"},
    {"$match": {"genre": genre}},
    {"$project": {"titre": 1, "duree": 1, "cout": 1, 
                  "cout_par_minute": {"$divide": ["$cout", {"$duree"}]}}}
    ]


    # Exécution de la requête et affichage des résultats
    results = collection.aggregate(pipeline)
    for result in results:
        print("Le coût de tournage d'une minute de film pour le genre {} est de ${:.2f}".format(result["titre"], result["cout_moyen_par_minute"]))


InvalidDocument: cannot encode object: {'$duree'}, of type: <class 'set'>

## Quelles sont les 5 séries les mieux notées ? 

In [98]:
# Changement de collection pour passer des films aux séries
collection = db['series']

top_series = collection.find(sort=[("score",-1)], limit=5)

print("Les 5 séries les mieux notées sont :")
for serie in top_series:
    print(f"{serie['Title']} avec un score de {serie['Score']} sur 10")


Les 5 séries les mieux notées sont :
Band of Brothers avec un score de 9.4 sur 10
The X-Files avec un score de 8.6 sur 10
Breaking Bad avec un score de 9.5 sur 10
Planet Earth II avec un score de 9.5 sur 10
Planet Earth avec un score de 9.4 sur 10
